#### Пример 13.1.Частичная реализация протокола последовательности – метода __getitem__

In [2]:
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]

v = Vowels()
v[0]

'A'

In [4]:
v[-1]

'U'

In [3]:
for c in v: 
    print(c)

A
E
I
O
U


In [5]:
'E' in v

True

In [6]:
'Z' in v

False

#### Пример 13.2. Колода как последовательность карт (тот же код, что в примере 1.1)

In [7]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

#### Партизанское латание как средство реализации протокола во время выполнения

In [8]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[6, 4, 7, 1, 8, 3, 5, 0, 2, 9]

#### Пример 13.3. random.shuffle не может работать с объектом FrenchDeck

In [9]:
from random import shuffle
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

#### Пример 13.4. Партизанское латание класса FrenchDeck с целью сделать его изменяемым и совместимым с функцией random.shuffle (продолжение примера 13.3)

In [10]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='K', suit='diamonds'),
 Card(rank='7', suit='hearts'),
 Card(rank='2', suit='spades'),
 Card(rank='Q', suit='clubs'),
 Card(rank='A', suit='diamonds')]

#### Пример 13.5. Применение утиной типизации для обработки строки или итерируемого объекта строк

In [11]:
try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass
field_names = tuple(field_names)
if not all(s.isidentifier() for s in field_names):
    raise ValueError('field_names must all be valid identifiers')

NameError: name 'field_names' is not defined

#### Пример 13.6. frenchdeck2.py: FrenchDeck2, подкласс collections.MutableSequence

In [1]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  # <1>
        self._cards[position] = value

    def __delitem__(self, position):  # <2>
        del self._cards[position]

    def insert(self, position, value):  # <3>
        self._cards.insert(position, value)

#### Пример 13.7. tombola.py: Tombola – ABC с двумя абстрактными и двумя конкретными методами

In [1]:
import abc

class Tombola(abc.ABC):  # <1>

    @abc.abstractmethod
    def load(self, iterable):  # <2>
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  # <3>
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  # <4>
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # <5>

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(items)

#### Пример 13.8. Непригодная реализация Tombola не останется незамеченной

In [5]:
class Fake(Tombola):
    def pick(self):
        return 13

Fake

__main__.Fake

In [6]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

#### Пример 13.9. bingo.py: BingoCage – конкретный подкласс Tombola

In [2]:
import random


class BingoCage(Tombola):  # <1>

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # <2>
        self._items = []
        self.load(items)  # <3>

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # <4>

    def pick(self):  # <5>
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  # <6>
        self.pick()

#### Пример 13.10. lotto.py: LotteryBlower – конкретный подкласс, в котором переопределены методы inspect и loaded ABC Tombola

In [3]:
import random


class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)  # <1>

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))  # <2>
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)  # <3>

    def loaded(self):  # <4>
        return bool(self._balls)

    def inspect(self):  # <5>
        return tuple(self._balls)

#### Пример 13.11. tombolist.py: TomboList – виртуальный подкласс Tombola

In [5]:
from random import randrange


@Tombola.register  # <1>
class TomboList(list):  # <2>

    def pick(self):
        if self:  # <3>
            position = randrange(len(self))
            return self.pop(position)  # <4>
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # <5>

    def loaded(self):
        return bool(self)  # <6>

    def inspect(self):
        return tuple(self)

# Tombola.register(TomboList)  # <7>

In [6]:
issubclass(TomboList, Tombola)

True

In [7]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [8]:
TomboList.__mro__

(__main__.TomboList, list, object)

In [9]:
class Struggle:
    def __len__(self): return 23

from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [10]:
issubclass(Struggle, abc.Sized)

True

#### Пример 13.12.Определениекласса SizedизфайлаLib/_collections_abc.py (https://github.com/python/cpython/blob/0fbddb14dc03f61738af01af88e7d8aa8df07336/Lib/_collections_abc.py#L369)

In [13]:
from abc import ABCMeta, abstractmethod

class Sized(metaclass=ABCMeta):
    
    __slots__ = ()
    
    @abstractmethod
    def __len__(self):
        return 0
        
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented

#### Типизированная функция double

In [15]:
def double(x):
    return x * 2

double(1.5)

3.0

In [16]:
double('A')

'AA'

In [17]:
double([10, 20, 30])

[10, 20, 30, 10, 20, 30]

In [18]:
from fractions import Fraction
double(Fraction(2, 5))

Fraction(4, 5)

#### Пример 13.13. double_protocol.py: определение double с использованием Protocol

In [19]:
from typing import TypeVar, Protocol

T = TypeVar('T')  # <1>

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ...  # <2>
        pass

RT = TypeVar('RT', bound=Repeatable)  # <3>

def double(x: RT) -> RT:  # <4>
    return x * 2

IndentationError: unexpected indent (2611681615.py, line 7)

#### Пример 13.14. Исходный код протокола typing.SupportsComplex

In [22]:
from typing import runtime_checkable, Protocol

@runtime_checkable
class SupportsComplex(Protocol):
    """ABC с одним абстрактным методом __complex__."""
    __slots__ = ()
    
    @abstractmethod
    def __complex__(self) -> complex:
        pass

#### Пример 13.15. Использование SupportsComplex во время выполнения

In [3]:
import numpy as np
from typing import SupportsComplex

c64 = np.complex64(3+4j)
isinstance(c64, complex)

False

In [4]:
isinstance(c64, SupportsComplex)

True

In [5]:
c = complex(c64)
c

(3+4j)

In [6]:
isinstance(c, SupportsComplex)

False

In [7]:
complex(c)

(3+4j)

In [8]:
isinstance(c, (complex, SupportsComplex))

True

In [9]:
import numbers
isinstance(c, numbers.Complex)

True

In [10]:
isinstance(c64, numbers.Complex)

True

#### Ограничения протоколов, допускающих проверку во время выполнения

In [11]:
import sys
sys.version

'3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]'

In [12]:
c = 3+4j
c.__float__

AttributeError: 'complex' object has no attribute '__float__'

In [13]:
c.__float__()

AttributeError: 'complex' object has no attribute '__float__'

In [15]:
from typing import SupportsFloat
c = 3+4j
isinstance(c, SupportsFloat)

False

In [16]:
issubclass(complex, SupportsFloat)

False

#### Пример 13.16. vector2d_v4.py: метод преобразования в complex и обратно

In [17]:
def __complex__(self):
    return complex(self.x, self.y)

@classmethod
def fromcomplex(cls, datum):
    return cls(datum.real, datum.imag)  # <1>

#### Пример 13.17. vector2d_v5.py: добавление аннотаций в интересующие нас методы

In [19]:
def __abs__(self) -> float:  # <1>
    return math.hypot(self.x, self.y)

def __complex__(self) -> complex:  # <2>
    return complex(self.x, self.y)

@classmethod
def fromcomplex(cls, datum: SupportsComplex) -> Vector2d:  # <3>
    c = complex(datum)  # <4>
    return cls(c.real, c.imag)

NameError: name 'Vector2d' is not defined

#### Пример 13.18. randompick.py: определение RandomPicker

In [20]:
from typing import Protocol, runtime_checkable, Any

@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...

#### Пример 13.19. randompick_test.py: использование RandomPicker

In [22]:
import random
from typing import Any, Iterable, TYPE_CHECKING

class SimplePicker:  # <2>
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> Any:  # <3>
        return self._items.pop()

def test_isinstance() -> None:  # <4>
    popper: RandomPicker = SimplePicker([1])  # <5>
    assert isinstance(popper, RandomPicker)  # <6>

def test_item_type() -> None:  # <7>
    items = [1, 2]
    popper = SimplePicker(items)
    item = popper.pick()
    assert item in items
    if TYPE_CHECKING:
        reveal_type(item)  # <8>
    assert isinstance(item, int)